# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 14 2022 8:30AM,246344,10,MSP213933,"Methapharm, Inc.",Released
1,Sep 14 2022 8:30AM,246345,10,8521818,"Snap Medical Industries, LLC",Released
2,Sep 14 2022 8:30AM,246345,10,8521819,"Snap Medical Industries, LLC",Released
3,Sep 14 2022 8:30AM,246345,10,8521820,"Snap Medical Industries, LLC",Released
4,Sep 14 2022 8:30AM,246345,10,8521821,"Snap Medical Industries, LLC",Released
5,Sep 14 2022 8:30AM,246345,10,SNAP31701559,"Snap Medical Industries, LLC",Released
6,Sep 14 2022 8:30AM,246345,10,SNAP98221,"Snap Medical Industries, LLC",Released
7,Sep 14 2022 8:30AM,246345,10,SNAP99221,"Snap Medical Industries, LLC",Released
8,Sep 14 2022 8:30AM,246345,10,SNAPG1157240,"Snap Medical Industries, LLC",Released
9,Sep 14 2022 8:30AM,246346,10,Enova-10632,Emerginnova,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
38,246344,Released,1
39,246345,Released,8
40,246346,Released,2
41,246347,Released,11
42,246349,Executing,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
246344,NaN,NaN,1.0
246345,NaN,NaN,8.0
246346,NaN,NaN,2.0
246347,NaN,NaN,11.0
246349,NaN,1.0,NaN


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
246065,0.0,0.0,1.0
246169,0.0,1.0,0.0
246178,0.0,0.0,1.0
246180,0.0,0.0,1.0
246182,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
246065,0,0,1
246169,0,1,0
246178,0,0,1
246180,0,0,1
246182,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,246065,0,0,1
1,246169,0,1,0
2,246178,0,0,1
3,246180,0,0,1
4,246182,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,246065,,,1
1,246169,,1,
2,246178,,,1
3,246180,,,1
4,246182,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 14 2022 8:30AM,246344,10,"Methapharm, Inc."
1,Sep 14 2022 8:30AM,246345,10,"Snap Medical Industries, LLC"
9,Sep 14 2022 8:30AM,246346,10,Emerginnova
11,Sep 14 2022 8:30AM,246347,10,ISDIN Corporation
22,Sep 14 2022 8:30AM,246349,10,"Senseonics, Incorporated"
23,Sep 13 2022 3:06PM,246335,10,RHL Supplies
24,Sep 13 2022 3:00PM,246334,16,Sartorius Bioprocess Solutions
31,Sep 13 2022 2:46PM,246201,21,"NBTY Global, Inc."
32,Sep 13 2022 2:44PM,246205,21,"NBTY Global, Inc."
33,Sep 13 2022 2:44PM,246203,21,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Sep 14 2022 8:30AM,246344,10,"Methapharm, Inc.",,,1
1,Sep 14 2022 8:30AM,246345,10,"Snap Medical Industries, LLC",,,8
2,Sep 14 2022 8:30AM,246346,10,Emerginnova,,,2
3,Sep 14 2022 8:30AM,246347,10,ISDIN Corporation,,,11
4,Sep 14 2022 8:30AM,246349,10,"Senseonics, Incorporated",,1,
5,Sep 13 2022 3:06PM,246335,10,RHL Supplies,,,1
6,Sep 13 2022 3:00PM,246334,16,Sartorius Bioprocess Solutions,,1,6
7,Sep 13 2022 2:46PM,246201,21,"NBTY Global, Inc.",,,1
8,Sep 13 2022 2:44PM,246205,21,"NBTY Global, Inc.",,,1
9,Sep 13 2022 2:44PM,246203,21,"NBTY Global, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 14 2022 8:30AM,246344,10,"Methapharm, Inc.",1,,
1,Sep 14 2022 8:30AM,246345,10,"Snap Medical Industries, LLC",8,,
2,Sep 14 2022 8:30AM,246346,10,Emerginnova,2,,
3,Sep 14 2022 8:30AM,246347,10,ISDIN Corporation,11,,
4,Sep 14 2022 8:30AM,246349,10,"Senseonics, Incorporated",,1,
5,Sep 13 2022 3:06PM,246335,10,RHL Supplies,1,,
6,Sep 13 2022 3:00PM,246334,16,Sartorius Bioprocess Solutions,6,1,
7,Sep 13 2022 2:46PM,246201,21,"NBTY Global, Inc.",1,,
8,Sep 13 2022 2:44PM,246205,21,"NBTY Global, Inc.",1,,
9,Sep 13 2022 2:44PM,246203,21,"NBTY Global, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 14 2022 8:30AM,246344,10,"Methapharm, Inc.",1,,
1,Sep 14 2022 8:30AM,246345,10,"Snap Medical Industries, LLC",8,,
2,Sep 14 2022 8:30AM,246346,10,Emerginnova,2,,
3,Sep 14 2022 8:30AM,246347,10,ISDIN Corporation,11,,
4,Sep 14 2022 8:30AM,246349,10,"Senseonics, Incorporated",,1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 14 2022 8:30AM,246344,10,"Methapharm, Inc.",1.0,NaN,NaN
1,Sep 14 2022 8:30AM,246345,10,"Snap Medical Industries, LLC",8.0,NaN,NaN
2,Sep 14 2022 8:30AM,246346,10,Emerginnova,2.0,NaN,NaN
3,Sep 14 2022 8:30AM,246347,10,ISDIN Corporation,11.0,NaN,NaN
4,Sep 14 2022 8:30AM,246349,10,"Senseonics, Incorporated",NaN,1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 14 2022 8:30AM,246344,10,"Methapharm, Inc.",1.0,0.0,0.0
1,Sep 14 2022 8:30AM,246345,10,"Snap Medical Industries, LLC",8.0,0.0,0.0
2,Sep 14 2022 8:30AM,246346,10,Emerginnova,2.0,0.0,0.0
3,Sep 14 2022 8:30AM,246347,10,ISDIN Corporation,11.0,0.0,0.0
4,Sep 14 2022 8:30AM,246349,10,"Senseonics, Incorporated",0.0,1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3202097,31.0,1.0,0.0
12,246329,1.0,0.0,0.0
15,246227,29.0,31.0,59.0
16,246334,6.0,1.0,0.0
19,492552,8.0,20.0,8.0
20,985232,3.0,7.0,23.0
21,3692870,14.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3202097,31.0,1.0,0.0
1,12,246329,1.0,0.0,0.0
2,15,246227,29.0,31.0,59.0
3,16,246334,6.0,1.0,0.0
4,19,492552,8.0,20.0,8.0
5,20,985232,3.0,7.0,23.0
6,21,3692870,14.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,31.0,1.0,0.0
1,12,1.0,0.0,0.0
2,15,29.0,31.0,59.0
3,16,6.0,1.0,0.0
4,19,8.0,20.0,8.0
5,20,3.0,7.0,23.0
6,21,14.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,31.0
1,12,Released,1.0
2,15,Released,29.0
3,16,Released,6.0
4,19,Released,8.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21
Status,,,,,,,
Completed,0.0,0.0,59.0,0.0,8.0,23.0,0.0
Executing,1.0,0.0,31.0,1.0,20.0,7.0,1.0
Released,31.0,1.0,29.0,6.0,8.0,3.0,14.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21
0,Completed,0.0,0.0,59.0,0.0,8.0,23.0,0.0
1,Executing,1.0,0.0,31.0,1.0,20.0,7.0,1.0
2,Released,31.0,1.0,29.0,6.0,8.0,3.0,14.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21
0,Completed,0.0,0.0,59.0,0.0,8.0,23.0,0.0
1,Executing,1.0,0.0,31.0,1.0,20.0,7.0,1.0
2,Released,31.0,1.0,29.0,6.0,8.0,3.0,14.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()